In [2]:
import numpy as np
from six import iteritems
from timeit import default_timer as timer 
from math import sqrt
import Call_Function as cf
import pickle

In [12]:
2**3

8

In [13]:
neigh=None
with open('similarity2.p','rb')as file:
    neigh=pickle.load(file)


In [24]:
neigh.overall_mean+neigh.bu[0]+neigh.bi[0]

2.1626370512073962

In [18]:
count=0
for i in range(neigh.n_movie):
    if neigh.sim_matrix[3,neigh.index_matrix[3,i]]!=0.0 and count<50:
        count+=1


In [114]:
np.nonzero(neigh.sim_matrix)

(array([], dtype=int64), array([], dtype=int64))

In [72]:
a=np.array([[1,2,3],[1,2,3],[1,2,3]])
b=np.array([[1,2,3],[2,3,4],[2,6,7]])
a[:,0]*b[:,0]


array([1, 2, 2])

In [75]:
c=(1,2,3)
a=np.asarray(c)
type(a)

numpy.ndarray

In [5]:
# Main Using Only Python
if __name__ == '__main__':
    n_user,n_movie,n_rating=cf.read_user_inform()
    #n_user : user 수
    #n_factor : factor 수
    #n_movie : item 수
    #epoch : epoch 수
    #train_set training 할 data set의 경로
    #num_nearest_k=150 : 근접 item 수
    #learning_rate=0.005 : learning rate 수
    #regulation=0.02 : regulation 
    #random_state=1 : seed로 난수로 정하기 위한 변수
    #lamda=100 : similarity 구할 때의 람다
    neigh=Neighbor_cor(n_user=n_user,n_factor=150,n_movie=n_movie,epoch=10,train_set=cf.data_location,num_nearest_k=30)
    # data 읽어 들이기
    neigh.read_u_data(cf.data_location)
    # Movie의 평균 구하기
    neigh.make_mean()
    # Movie의 similarity 구하기
    neigh.make_sim_matrix()
    neigh.make_top_sim_matrix()
    RMSE_list=[]
    start=timer()
    print('CorMvg Start')
    for i in range(cf.k_fold):
        neigh.gradient_descent(cf.train_set_list[i])
        RMSE=neigh.predict(cf.test_set_list[i])
        print(RMSE)
        RMSE_list.append(RMSE)
    time=timer()-start
    print("Time : ",time)
    avg_RMSE=sum(RMSE_list)/len(RMSE_list)
    print("AVG RMSE : ",avg_RMSE)

Read Start
Finish
Get All Mean , Time :  2.6804644789081067


/home/kibum/anaconda3/envs/kibum/lib/python3.7/site-packages/ipykernel_launcher.py:73: RuntimeWarning: invalid value encountered in double_scalars


KeyboardInterrupt: 

In [4]:

class Neighbor_cor:
    #train_set indicate the route of train set
    def __init__(self,n_user,n_factor,n_movie,epoch,train_set,num_nearest_k=150,learning_rate=0.005,regulation=0.02,random_state=1,lamda=100):
        self.n_user=n_user
        self.n_factor=n_factor
        self.n_movie=n_movie
        self.rating_matrix=np.zeros((n_user,n_movie),np.int8)
        self.movie_mean=np.zeros(n_movie,np.float32)
        self.sim_matrix=np.zeros((self.n_movie,self.n_movie))
        self.index_matrix=None
        #default value - 100
        self.lamda=lamda
        self.overall_mean=0
        self.epochs=epoch
        #default value - 1
        self.random_state=random_state
        self.lr=learning_rate
        self.reg=regulation
        self.bu=None
        self.bi=None
        self.k=num_nearest_k
        self.train_set=train_set
    # Reading total data for entering rating to matrix    
    def read_u_data(self,location):
        with open(location,'r') as f:
            print("Read Start")
            line=f.readline().split('\t')
            while len(line)>1:
                self.rating_matrix[int(line[0])-1,int(line[1])-1]=int(line[2])
                line=f.readline().split('\t')
            print("Finish")
            
    # get overall mean and each movie mean        
    def make_mean(self):
        start=timer()
        total_count=0
        total_sum=0
        for j in range(self.n_movie):
            sum=0
            count=0
            for i in range(self.n_user):
                if self.rating_matrix[i,j] != 0:
                    sum+=self.rating_matrix[i,j]
                    total_count+=1
                    total_sum+=self.rating_matrix[i,j]
                    count+=1
            self.movie_mean[j]=sum/count
        self.overall_mean=total_sum/total_count
        print("Get All Mean , Time : ",timer()-start)
    
    #Make Similarity Matrix
    def make_sim_matrix(self):
        start=timer()
        #loading=0
        for i in range(self.n_movie):
            for j in range(self.n_movie):
                if i == j:
                    self.sim_matrix[i][j]=0
                    continue
                x_sum=0
                y_sum=0
                xy_sum=0
                count=0
                for k in range(self.n_user):                 
                    if self.rating_matrix[k][i] != 0 and self.rating_matrix[k][j] != 0:
                        count+=1
                        x_sum+=(self.rating_matrix[k][i]-self.movie_mean[i])**2
                        y_sum+=(self.rating_matrix[k][j]-self.movie_mean[j])**2
                        xy_sum+=(self.rating_matrix[k][i]-self.movie_mean[i])*(self.rating_matrix[k][j]-self.movie_mean[j]) 
                if x_sum==0:
                    self.sim_matrix[i][j]=self.sim_matrix[j][i]=0
                else:   
                    self.sim_matrix[i][j]=self.sim_matrix[j][i]=(count/(count+self.lamda))*(xy_sum/(sqrt(x_sum*y_sum)))
                #loading+=1
                #print('Loading : {}\r'.format(loading), end='')
        self.rating_matrix=rating_matrix
        self.sim_matrix=sim_matrix
        print("End Time : ",timer()-start)
    
    # Training
    def gradient_descent(self,random):
        k=self.k
        bu=np.zeros(self.n_user,np.int32)
        bi=np.zeros(self.n_movie,np.int32)
        # Make index matrix of sorted similarity
        index_matrix=make_top_sim_matrix()
        for epoch in range(self.epochs):
            with open(self.train_set,'r') as file:
                line=file.readline().split('\t')
                #sum_top is sum of s_ij*(r_uj-b_uj) in S_k(i;u)
                sum_top=0
                #sum_bottom is sum of s_ij in S_k(i;u)
                sum_bottom=0
                while len(line) >1:
                    #number of movie id relating to train_set
                    movie_index=line[1]-1
                    user_index=line[0]-1
                    k_=0
                    index=0
                    while k_<k:
                        _index=index_matrix[movie_index][index]
                        if self.rating_matrix[user_index][_index] is not 0:
                            baseline=self.overall_mean+bu[user_index]+bi[movie_index]
                            sum_top+=self.sim_matrix[movie_index][_index]*(self.rating_matrix[user_index][_index]-baseline)
                            sum_bottom+=self.sim_matrix[movie_index][_index]
                            k_+=1
                        index+=1
                    r_ui_hat=self.overall_mean+bu[user_index]+bi[movie_index]+sum_top/sum_bottom
                    error=self.rating_matrix[user_index][movie_index]-r_ui_hat
                    bu[user_index]=bu[user_index]+self.lr*(error-self.reg*bu[user_index])
                    bi[movie_index]=bi[movie_index]+self.lr*(error-self.reg*bi[movie_index])
                    line=file.readline().split('\t')
        
        self.bi=bi
        self.bu=bu
        return 0
    
     #Testing
    def predict(self,test_set):
        err_sum=0
        total_count=0
        with open(test_set,'r') as file:
            line=file.readline().split('\t')
            while len(line) >1:
                total_count+=1
                #number of movie id relating to train_set
                movie_index=int(line[1])-1
                user_index=int(line[0])-1
                rating=int(line[2])
                k_=0
                index=0
                #sum_top is sum of s_ij*(r_uj-b_uj) in S_k(i;u)
                sum_top=0
                #sum_bottom is sum of s_ij in S_k(i;u)
                sum_bottom=0
                while k_<self.k:
                    _index=self.index_matrix[movie_index,index]
                    if self.sim_matrix[movie_index,_index]==0:
                        break
                    if self.rating_matrix[user_index,_index] != 0:
                        baseline=overall_mean+self.bu[user_index]+self.bi[_index]
                        sum_top+=self.sim_matrix[movie_index,_index]*(self.rating_matrix[user_index,_index]-baseline)
                        sum_bottom+=self.sim_matrix[movie_index,_index]
                        k_+=1
                    
                    index+=1
                plus=sum_top/sum_bottom if sum_bottom!=0 else 0
                r_ui_hat=overall_mean+self.bu[user_index]+self.bi[movie_index]+plus
                error=self.rating_matrix[user_index,movie_index]-r_ui_hat
                err_sum+=(error)**2
                line=file.readline().split('\t')
        return sqrt(err_sum/total_count)
    
    # Make Index Matrix of descending order of similarity                
    def make_top_sim_matrix(self):
        index_matrix=[]
        for i in range(self.n_user):
            sort_index=self.sim_matrix[i].argsort()[::-1]
            index_matrix.append(sort_index)
        self.index_matrix=np.array(index_matrix)
        return index_matrix
        